In [1]:
import rioxarray
import geopandas
import os
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
out_dir = "data/_grace_tiff"
os.makedirs(out_dir, exist_ok=True)
geodf = geopandas.read_file("data/aoi.geojson")
nc = rioxarray.open_rasterio("data/GRCTellus.JPL.200204_202202.GLO.RL06M.MSCNv02CRI.nc", decode_times=False)[0]
grace_start_date = datetime.datetime(2002, 1, 1)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rioxarray/_io.py:1132: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rioxarray/_io.py:1132: NotGeoreferencedWarning: Dat

In [20]:
nc = nc.rio.write_crs("EPSG:4326")
print("Extracting tifs...")
df = pd.DataFrame(columns = ["date", "value"])
for lwe in tqdm(nc["lwe_thickness"]):
    lwe = lwe.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    lwe = lwe.rio.write_crs("EPSG:4326")
    new_width = 13
    new_height = 8

    lwe_upsampled = lwe.rio.reproject(
        lwe.rio.crs,
        shape=(new_height, new_width),
    )
    lwe_datetime = grace_start_date + datetime.timedelta(days=int(lwe.time.values))
    lwe_upsampled.values[lwe_upsampled.values < -99990] = np.nan
    mean = lwe_upsampled.mean(skipna=True)
    masked = lwe_upsampled.to_masked_array()
    df = df.append({'date': lwe_datetime.strftime('%Y-%m') + "-01", 'value': masked.data}, ignore_index=True)

Extracting tifs...


  0%|          | 0/206 [00:00<?, ?it/s]/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_2052/3269957774.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': lwe_datetime.strftime('%Y-%m') + "-01", 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_2052/3269957774.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': lwe_datetime.strftime('%Y-%m') + "-01", 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_2052/3269957774.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': lwe_datetime.strftime('%Y-%m') + "-01", 'value': masked.data}, ignore_inde

In [21]:
df.loc[df["date"] == "2012-01-01"]
df.iloc[111]["date"] = "2011-12-01"
df.iloc[144]["date"] = "2015-05-01"
df.to_pickle("data/grace_df.pickle")